<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/DeepLearning/3_Redes_Multicapa/ejercicios/ejercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Ejercicios Clase 3

En este notebook vamos a usar MLPs para generar un modelo clasificador sobre FashionMNIST así que muchas de las funciones que usamos en los ejercicios de la clase 2 te serán muy útiles.

## Ejercicio 1:

Generar un modelo perceptron multicapa con 2 capas ocultas de 512 y 128 neuronas respectivamente para clasificación sobre el dataset FashionMNIST

In [27]:
import torch
from torch import nn
import torchvision
#from IPython import display
from torchvision import transforms
from torch.utils import data


net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 512),
                    nn.ReLU(),
                    nn.Linear(512, 128),
                    nn.ReLU(),
                    nn.Linear(128, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

## Ejercicio 2

Entrene el modelo por 10 épocas con un tamaño de lote de 256 y un learning rate de 0.3. (Le recomendamos reutilizar las funciones modularizadas de los ejercicios de la clase 2)

In [28]:
#ingresa tu código aquí
batch_size, lr, num_epochs = 256, 0.3, 10
loss = nn.CrossEntropyLoss(reduction='none')
trainer = torch.optim.SGD(net.parameters(), lr=lr)


In [29]:
def load_data_fashion_mnist(batch_size, resize=None):
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=1),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=1))

In [30]:
def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [31]:
train_iter, test_iter = load_data_fashion_mnist(batch_size)

In [32]:
def train_epoch(train_iter,net,loss_fn, optimizer,accuracy_fn, trackers):
    for X, y in train_iter:
        optimizer.zero_grad()
        y_hat = net(X)
        l = loss_fn(y_hat, y)
        l.mean().backward()
        optimizer.step()
        with torch.no_grad():
            trackers['epoch_loss'] += l.sum()
            trackers['epoch_acc'] += accuracy_fn(y_hat, y)
            trackers['num_samples'] += len(X)
    

def test_epoch(test_iter,net,accuracy_fn, trackers):
    for X, y in test_iter:
        with torch.no_grad():
            y_hat = net(X)
            trackers['test_acc'] += accuracy_fn(y_hat, y)
            trackers['num_test_samples'] += len(X)
            

In [33]:
def train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy):
    trackers = {'epoch_loss': 0.,
                'epoch_acc': 0.,
                'num_samples': 0.,
                'test_acc': 0.,
                'num_test_samples': 0.
                }
    
    for epoch in range(num_epochs):
        trackers['epoch_loss'] = 0.
        trackers['epoch_acc'] = 0.
        trackers['num_samples'] = 0.
        trackers['test_acc'] = 0.
        trackers['num_test_samples'] = 0.
        train_epoch(train_iter,net,loss, trainer,accuracy, trackers)
        test_epoch(test_iter,net,accuracy, trackers)
        print(f'Epoch {epoch+1}: loss {trackers["epoch_loss"]/trackers["num_samples"]:.3f}, accuracy {trackers["epoch_acc"]/trackers["num_samples"]:.3f}, test accuracy {trackers["test_acc"]/trackers["num_test_samples"]:.3f}')


In [34]:
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe18d67db70>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'


AssertionError: can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe20dafee80>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe18d67db70>>
Traceback (most recent call last):
  Fil

Epoch 1: loss 1.180, accuracy 0.541, test accuracy 0.756


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe18d67db70>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe20dafee80>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/pyt

Epoch 2: loss 0.567, accuracy 0.790, test accuracy 0.779


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe18d67db70>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe20dafee80>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/pyt

## Ejercicio 3 :

A partir del modelo anterior, analice que ocurre si en lugar de entrenar 10 épocas, entrena 20

In [ ]:
#ingresa tu código aquí
num_epochs = 20
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)


Epoch 1: loss 0.229, accuracy 0.914, test accuracy 0.748
Epoch 2: loss 0.234, accuracy 0.914, test accuracy 0.869
Epoch 3: loss 0.217, accuracy 0.918, test accuracy 0.872
Epoch 4: loss 0.210, accuracy 0.920, test accuracy 0.839
Epoch 5: loss 0.205, accuracy 0.922, test accuracy 0.874
Epoch 6: loss 0.203, accuracy 0.923, test accuracy 0.871
Epoch 7: loss 0.194, accuracy 0.926, test accuracy 0.877
Epoch 8: loss 0.192, accuracy 0.927, test accuracy 0.882
Epoch 9: loss 0.187, accuracy 0.930, test accuracy 0.872
Epoch 10: loss 0.186, accuracy 0.929, test accuracy 0.883
Epoch 11: loss 0.178, accuracy 0.933, test accuracy 0.883
Epoch 12: loss 0.179, accuracy 0.932, test accuracy 0.886
Epoch 13: loss 0.193, accuracy 0.930, test accuracy 0.867
Epoch 14: loss 0.186, accuracy 0.930, test accuracy 0.868
Epoch 15: loss 0.169, accuracy 0.937, test accuracy 0.881
Epoch 16: loss 0.168, accuracy 0.936, test accuracy 0.886
Epoch 17: loss 0.165, accuracy 0.937, test accuracy 0.877
Epoch 18: loss 0.161, a

## Ejercicio 4

Aumente el learning rate a 1 y entrene nuevamente. ¿Cómo puede explicar lo que pasó?

In [ ]:
lr = 1 
trainer = torch.optim.SGD(net.parameters(), lr=lr)
num_epochs = 20
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)

Epoch 1: loss 5597757875809036592480256.000, accuracy 0.110, test accuracy 0.100
Epoch 2: loss 2.304, accuracy 0.099, test accuracy 0.100
Epoch 3: loss 2.304, accuracy 0.101, test accuracy 0.100
Epoch 4: loss 2.304, accuracy 0.100, test accuracy 0.100
Epoch 5: loss 2.304, accuracy 0.097, test accuracy 0.100
Epoch 6: loss 2.304, accuracy 0.100, test accuracy 0.100
Epoch 7: loss 2.304, accuracy 0.101, test accuracy 0.100
Epoch 8: loss 2.304, accuracy 0.102, test accuracy 0.100
Epoch 9: loss 2.304, accuracy 0.098, test accuracy 0.100
Epoch 10: loss 2.303, accuracy 0.100, test accuracy 0.100
Epoch 11: loss 2.304, accuracy 0.099, test accuracy 0.100
Epoch 12: loss 2.304, accuracy 0.101, test accuracy 0.100
Epoch 13: loss 2.304, accuracy 0.098, test accuracy 0.100
Epoch 14: loss 2.304, accuracy 0.100, test accuracy 0.100
Epoch 15: loss 2.303, accuracy 0.101, test accuracy 0.100
Epoch 16: loss 2.304, accuracy 0.099, test accuracy 0.100
Epoch 17: loss 2.304, accuracy 0.099, test accuracy 0.100

## Ejercicio 5:

Analize el efecto de cambiar las funciones de activación en el accurracy

In [ ]:
#ingresa tu código aquí
net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 512),
                    nn.Tanh(),
                    nn.Linear(512, 128),
                    nn.Tanh(),
                    nn.Linear(128, 10))

lr= 0.1
trainer = torch.optim.SGD(net.parameters(), lr=lr)
num_epochs = 10
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)

Epoch 1: loss 0.866, accuracy 0.714, test accuracy 0.768
Epoch 2: loss 0.528, accuracy 0.811, test accuracy 0.785
Epoch 3: loss 0.478, accuracy 0.828, test accuracy 0.814
Epoch 4: loss 0.447, accuracy 0.839, test accuracy 0.806
Epoch 5: loss 0.434, accuracy 0.843, test accuracy 0.821
Epoch 6: loss 0.414, accuracy 0.850, test accuracy 0.833
Epoch 7: loss 0.404, accuracy 0.855, test accuracy 0.844
Epoch 8: loss 0.392, accuracy 0.859, test accuracy 0.848
Epoch 9: loss 0.385, accuracy 0.862, test accuracy 0.835
Epoch 10: loss 0.375, accuracy 0.865, test accuracy 0.848


## Ejercicio 6:

Ahora genere un tercer modelo en donde ambas capas tengan 1024 neuronas. Analice si produjo algún cambio en los rendimientos.

In [ ]:
#ingresa tu código aquí
et = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 1024),
                    nn.ReLU(),
                    nn.Linear(1024, 1024),
                    nn.ReLU(),
                    nn.Linear(1024, 10))

lr= 0.01
trainer = torch.optim.SGD(net.parameters(), lr=lr)
num_epochs = 20
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe18d67db70>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe20dafee80>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/pyt

Epoch 1: loss 0.282, accuracy 0.897, test accuracy 0.875


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe18d67db70>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe20dafee80>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/pyt

Epoch 2: loss 0.282, accuracy 0.898, test accuracy 0.876


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe18d67db70>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe20dafee80>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/pyt

Epoch 3: loss 0.281, accuracy 0.898, test accuracy 0.877


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe18d67db70>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe20dafee80>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/pyt

Epoch 4: loss 0.281, accuracy 0.898, test accuracy 0.875


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe18d67db70>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe20dafee80>>
Traceback (most recent call last):
  File "/home/vfs/anaconda3/envs/env_pytorch/lib/pyt

KeyboardInterrupt: 